<a href="https://colab.research.google.com/github/MasahiroAraki/MachineLearning3/blob/master/notebook/chap02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 60.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.39.0 requires scikit-learn<1.8.0,>=0.24, but you have scikit-learn 1.8.0 which is incompatible.


# 2. 機械学習の基本的な手順


## 2.1 Pythonによる機械学習の実装


In [2]:
# 2章で用いるライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline

In [3]:
# Jupyter notebook で出力を見やすくするための設定
%precision 3
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.precision', 3)
sns.set_theme()

## 2.2 データ収集・整理

- [load_iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)メソッドでirisデータセットを読み込む
  - as_frame: Trueのとき、data属性がpandasのDataFrame型に、target属性がpandasのSeries型になる

- irisデータセット
  - アヤメ (iris) の種類を，その萼（がく）の長さ (sepal length) ・幅 (sepal width) ，花びらの長さ (petal length) ・幅 (petal width) の，計四つの特徴を用いて識別するための学習データ
  - 各事例には，0 (Iris-setosa), 1 (Iris-versicolor), 2 (Iris-virginica) のいずれかが正解情報として付いている


In [4]:
iris = load_iris(as_frame=True)
X = iris.data
y = iris.target
print(iris.DESCR) # データセットの詳細の表示

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [5]:
# データに関する情報の表示
print(f"Shape of the dataset: {X.shape}")
print(f"Feature names: {iris.feature_names}")
print(f"Shape of the target: {y.shape}")
print(f"Target class names: {iris.target_names}")

Shape of the dataset: (150, 4)
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Shape of the target: (150,)
Target class names: ['setosa' 'versicolor' 'virginica']


## 2.3 探索的データ解析

データ分析用には、正解情報も含めてひとつの DataFrame として、seaborn を用いる。

In [6]:
df = sns.load_dataset("iris")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [7]:
# 統計情報を表示
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000,150.000,150.000,150.000
mean,5.843,3.057,3.758,1.199
std,0.828,0.436,1.765,0.762
min,4.300,2.000,1.000,0.100
25%,5.100,2.800,1.600,0.300
50%,5.800,3.000,4.350,1.300
75%,6.400,3.300,5.100,1.800
max,7.900,4.400,6.900,2.500


In [ ]:
# 対角成分は各特徴の分布、非対角成分は2つの特徴の散布図
# いずれもクラス別に表示
markers = ['o', '^', 's']
sns.pairplot(df, hue="species", markers=markers)
plt.savefig('s2-1.svg', format='svg', bbox_inches='tight') # svg出力
plt.show()

## 2.4 前処理

正規化 [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
print('mean:', np.mean(X_scaled, axis=0))
print('std :', np.std(X_scaled, axis=0))

## 2.5 評価基準の設定と学習

これ以降のコーディングにあたり、評価法を決めておく

- 分割学習法
  - 学習の前にデータを学習用と評価用に分割する
- 交差確認法
  - 学習時に交差確認を行うメソッドを呼び出す


## 学習

[k-NN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)による識別を行う

### 分割学習法

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=7) # test_size: テストデータの割合
print(f'train data: {len(X_train)} class: {np.bincount(y_train)}')
print(f'test data : {len(X_test)}  class: {np.bincount(y_test)}')

In [ ]:
clf = KNeighborsClassifier(n_neighbors=3) # n_neighbors: 参照する近傍のデータ数
print(clf)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = sum(y_pred == y_test) / len(y_test)
print(f'Accuracy: {acc:0.2f}')

### 交差確認法

In [ ]:
# 精度の平均値と95%信頼区間の幅（標準偏差の2倍）を計算
scores = cross_val_score(clf, X_scaled, y, cv=10) # StratifiedKFold, ランダム性なし
print(f'Accuracy: {scores.mean():0.2f} (+/- {scores.std()*2:0.2f})')

## パイプライン

上記手順は標準化の際にテストデータも一緒にしてしまっているので、厳密には学習データとテストデータの分離が不十分である。前処理と学習をパイプラインでまとめてひとつの識別器とし、この識別器に対して交差確認を行うべきである。

In [ ]:
pipe = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier()
    )

# 分割学習法の場合
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)
score_div = pipe.fit(X_train, y_train).score(X_test, y_test)
print(f'Split Accuracy: {score_div:0.2f}')

# 交差確認法の場合
score_cross = cross_val_score(pipe, X, y, cv=10)
print(f'Cross-val Accuracy: {score_cross.mean():0.2f} (+/- {score_cross.std()*2:0.2f})')

In [ ]:
# パイプラインの構成の表示
pipe

In [ ]:
# 評価と混同行列の出力
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# 混同行列の計算
cm = confusion_matrix(y_test, y_pred, labels=pipe.classes_)
#df_cm = pd.DataFrame(cm, index=iris.target_names, columns=iris.target_names)

# ヒートマップの作成
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.ylabel('True label')
plt.xlabel('Predicted label')
#plt.title('Confusion Matrix')
plt.savefig('s2-2.svg', format='svg', bbox_inches='tight')
plt.show()

## 参考

* PCA https://scikit-learn.org/stable/modules/decomposition.html#pca
* Cross validation https://scikit-learn.org/stable/modules/cross_validation.html
* kNN https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
* Metrics https://scikit-learn.org/stable/modules/model_evaluation.html
* Pipeline https://scikit-learn.org/stable/modules/compose.html#combining-estimators

## 演習問題


### 2-1

Breast Cancer データに対して，k-NN 法で識別を行うときに、適切な近傍データ数（k の値）を求めるコードを作成してください。

In [ ]:
from sklearn.datasets import load_breast_cancer

# データのロード
data = load_breast_cancer()
X, y = data.data, data.target

# データを学習用と評価用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

# 近傍数の候補を定義
neighbor_candidates = range(1, 11)
best_score = -np.inf
best_k = None

# 各近傍数に対して交差確認を実施
for k in neighbor_candidates:
    pipeline = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=k))
    # 5分割の交差確認を実施
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    mean_score = scores.mean()
    print(f"k = {k}: Cross-validation accuracy = {mean_score:.3f}")

    # 最良のスコアと対応するkを更新
    if mean_score > best_score:
        best_score = mean_score
        best_k = k

# 最適な近傍数とそのときのスコアを表示
print(f"\nBest number of neighbors: {best_k}")
print(f"Best cross-validation score: {best_score:.3f}")

# 最適な近傍数でパイプラインを再構築し、学習と評価
best_pipeline = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=best_k))
best_pipeline.fit(X_train, y_train)
print(f"Test set accuracy: {best_pipeline.score(X_test, y_test):.3f}")

### 2-2
scikit-learn で計算した混同行列から，正解率・適合率・再現率・F1値を計算するコードを作成してください．

In [ ]:
from sklearn.metrics import confusion_matrix

def performance_metrics(y_true, y_pred):
    # 混同行列を計算
    cm = confusion_matrix(y_true, y_pred)
    # True Positive, False Positive, True Negative, False Negative
    TP = cm[1, 1]
    FP = cm[0, 1]
    TN = cm[0, 0]
    FN = cm[1, 0]

    # 正解率 (Accuracy)
    accuracy = (TP + TN) / float(TP + TN + FP + FN)

    # 適合率 (Precision)
    precision = TP / float(TP + FP)

    # 再現率 (Recall or Sensitivity)
    recall = TP / float(TP + FN)

    # F値 (F1 Score)
    f1_score = 2 * precision * recall / (precision + recall)

    return accuracy, precision, recall, f1_score, cm

# テストデータ (例)
y_true = [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 0, 0]

accuracy, precision, recall, f1_score, cm = performance_metrics(y_true, y_pred)

# 結果の表示
print("Confusion Matrix:")
print(cm)
print(f"Accuracy:  {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1_score:.3f}")